UnicodeDecodeError: 'utf-8' codec can't decode byte 0x97 in position 1560: invalid start byte

In [ ]:
from io import StringIO

# Convert timestamp to datetime
events['date'] = pd.to_datetime(events['timestamp'], unit='s')

# Filter to your date range
start_date = pd.Timestamp('2022-04-28')
end_date = pd.Timestamp('2022-05-25')
events_filtered = events[(events['date'] >= start_date) & (events['date'] <= end_date)].copy()

# Create timeline plot
fig, ax = plt.subplots(figsize=(16, 10))

# Assign y-positions based on type and stablecoin
y_positions = {
    ('negative', 'ustc'): 1,
    ('negative', 'lunaterra'): 2,
    ('positive', 'ustc'): 3,
    ('positive', 'usdc'): 4,
    ('positive', 'usdt'): 5,
    ('negative', 'usdc'): 6,
}

# Color coding
colors = {
    'positive': 'green',
    'negative': 'red'
}

markers = {
    'ustc': 'o',
    'lunaterra': 's',
    'usdc': '^',
    'usdt': 'D',
    'dai': 'v',
    'unknown': 'x'
}

# Plot each event
for idx, row in events_filtered.iterrows():
    y_pos = y_positions.get((row['type'], row['stablecoin']), 3.5)
    
    ax.scatter(row['date'], y_pos, 
               color=colors[row['type']], 
               marker=markers.get(row['stablecoin'], 'o'),
               s=200, 
               alpha=0.7,
               edgecolors='black',
               linewidth=1.5,
               zorder=5)
    
    # Add event label
    ax.annotate(row['event'][:50] + '...' if len(row['event']) > 50 else row['event'],
                xy=(row['date'], y_pos),
                xytext=(10, 0),
                textcoords='offset points',
                fontsize=8,
                bbox=dict(boxstyle='round,pad=0.3', facecolor=colors[row['type']], alpha=0.2),
                ha='left',
                va='center')

# Mark May 9 crisis
ax.axvline(x=pd.Timestamp('2022-05-09'), color='black', linestyle='--', linewidth=3, alpha=0.7, label='Crisis Start')

# Formatting
ax.set_yticks([1, 2, 3, 4, 5, 6])
ax.set_yticklabels(['UST Negative', 'LUNA Negative', 'UST Positive', 'USDC Positive', 'USDT Positive', 'USDC Negative'])
ax.set_xlabel('Date', fontsize=14, fontweight='bold')
ax.set_ylabel('Event Category', fontsize=14, fontweight='bold')
ax.set_title('Terra-Luna Collapse: Timeline of Events (April 28 - May 25, 2022)', 
             fontsize=16, fontweight='bold', pad=20)
ax.grid(True, alpha=0.3, axis='x')

# Format x-axis
ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
ax.xaxis.set_major_locator(mdates.DayLocator(interval=2))
plt.xticks(rotation=45, ha='right')

# Legend
from matplotlib.lines import Line2D
legend_elements = [
    Line2D([0], [0], marker='o', color='w', markerfacecolor='green', markersize=10, label='Positive Event'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='red', markersize=10, label='Negative Event'),
    Line2D([0], [0], marker='o', color='w', markerfacecolor='gray', markersize=10, label='UST'),
    Line2D([0], [0], marker='s', color='w', markerfacecolor='gray', markersize=10, label='LUNA'),
    Line2D([0], [0], marker='^', color='w', markerfacecolor='gray', markersize=10, label='USDC'),
    Line2D([0], [0], linestyle='--', color='black', linewidth=2, label='May 9 Crisis'),
]
ax.legend(handles=legend_elements, loc='upper left', fontsize=10)

plt.tight_layout()
plt.show()